# Introduction
Dự án này sử dụng mô hình CNN-LSTM để phân loại 5 tư thế yoga (Bhujasana, Padamasana, Tadasana, Trikasana, Vrikshasana) từ dữ liệu video. CNN (ResNet50) trích xuất 2048 đặc trưng mỗi frame, trong khi LSTM học mối quan hệ thời gian giữa 16 frame mỗi clip. Mục tiêu là xây dựng hệ thống nhận diện tư thế với độ chính xác cao. Báo cáo được thực hiện vào ngày 18/06/2025.

In [7]:
# Data Preparation
import numpy as np
import json
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import logging
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Thiết lập logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - [LSTM] %(message)s', handlers=[
    logging.FileHandler('training.log', encoding='utf-8'),
    logging.StreamHandler()
])

# Dataset tùy chỉnh
class VideoDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features  # (num_samples, 16, 2048)
        self.labels = labels      # (num_samples,)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return torch.tensor(self.features[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.long)

# Load dữ liệu
def load_data(npz_path):
    data = np.load(npz_path)
    features = data['data']     # (num_samples, 16, 2048)
    labels = data['labels']     # (num_samples,)
    data.close()
    return features, labels

# Load ánh xạ nhãn
def load_label_map(json_path):
    with open(json_path, 'r', encoding='utf-8') as f:
        label2id = json.load(f)
    id2label = {str(v): k for k, v in label2id.items()}
    return id2label, len(label2id)

# Load và kiểm tra dữ liệu train
train_features, train_labels = load_data("processed_data/results_cnn_lstm/final_dataset.npz")
id2label, num_classes = load_label_map("processed_data/results_cnn_lstm/final_dataset_labels.json")
unique_train_labels, train_counts = np.unique(train_labels, return_counts=True)
train_distribution = {id2label[str(label)]: count for label, count in zip(unique_train_labels, train_counts)}

# Load và kiểm tra dữ liệu test
test_features, test_labels = load_data("processed_data/results_cnn_lstm_test/final_dataset_test.npz")
unique_test_labels, test_counts = np.unique(test_labels, return_counts=True)
test_distribution = {id2label[str(label)]: count for label, count in zip(unique_test_labels, test_counts)}

# In thông tin
print("=== Thông tin tập train ===")
print(f"Số mẫu: {len(train_labels)}")
print(f"Kích thước mỗi mẫu: {train_features.shape[1:]} (frame, features)")
print(f"Phân bố nhãn: {train_distribution}")

print("\n=== Thông tin tập test ===")
print(f"Số mẫu: {len(test_labels)}")
print(f"Kích thước mỗi mẫu: {test_features.shape[1:]} (frame, features)")
print(f"Phân bố nhãn: {test_distribution}")


=== Thông tin tập train ===
Số mẫu: 1635
Kích thước mỗi mẫu: (16, 2048) (frame, features)
Phân bố nhãn: {'Bhujasana': 327, 'Padamasana': 303, 'Tadasana': 345, 'Trikasana': 300, 'Vrikshasana': 360}

=== Thông tin tập test ===
Số mẫu: 407
Kích thước mỗi mẫu: (16, 2048) (frame, features)
Phân bố nhãn: {'Bhujasana': 99, 'Padamasana': 87, 'Tadasana': 90, 'Trikasana': 87, 'Vrikshasana': 44}


In [8]:
# Model Training
# Mô hình LSTM
class LSTMClassifier(nn.Module):
    def __init__(self, input_size=2048, hidden_size=512, num_layers=2, num_classes=5, dropout=0.3):
        super(LSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        _, (h_n, _) = self.lstm(x)  # h_n: (num_layers, batch_size, hidden_size)
        out = self.fc(h_n[-1])     # Lấy hidden state của lớp cuối
        return out

# Hàm huấn luyện và đánh giá
def train_and_evaluate(train_features, train_labels, test_features, test_labels, id2label, num_epochs=20, batch_size=32):
    # Chuẩn bị dữ liệu
    train_dataset = VideoDataset(train_features, train_labels)
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_dataset = VideoDataset(test_features, test_labels)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

    # Khởi tạo mô hình
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = LSTMClassifier(num_classes=len(id2label)).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Lưu trữ loss
    train_losses = []

    # Huấn luyện
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for batch_features, batch_labels in train_dataloader:
            batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
            optimizer.zero_grad()
            outputs = model(batch_features)
            loss = criterion(outputs, batch_labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(train_dataloader)
        train_losses.append(avg_loss)
        logging.info(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_loss:.4f}")

    return model, train_losses, train_dataloader, test_dataloader, device

# Chạy huấn luyện
model, train_losses, train_dataloader, test_dataloader, device = train_and_evaluate(train_features, train_labels, test_features, test_labels, id2label)


2025-11-11 00:15:19,869 - INFO - [LSTM] Epoch 1/20, Train Loss: 0.2518
2025-11-11 00:15:21,560 - INFO - [LSTM] Epoch 2/20, Train Loss: 0.0022
2025-11-11 00:15:23,221 - INFO - [LSTM] Epoch 3/20, Train Loss: 0.0006
2025-11-11 00:15:24,888 - INFO - [LSTM] Epoch 4/20, Train Loss: 0.0003
2025-11-11 00:15:26,562 - INFO - [LSTM] Epoch 5/20, Train Loss: 0.0002
2025-11-11 00:15:28,223 - INFO - [LSTM] Epoch 6/20, Train Loss: 0.0002
2025-11-11 00:15:29,914 - INFO - [LSTM] Epoch 7/20, Train Loss: 0.0001
2025-11-11 00:15:31,537 - INFO - [LSTM] Epoch 8/20, Train Loss: 0.0001
2025-11-11 00:15:33,164 - INFO - [LSTM] Epoch 9/20, Train Loss: 0.0001
2025-11-11 00:15:34,799 - INFO - [LSTM] Epoch 10/20, Train Loss: 0.0001
2025-11-11 00:15:36,450 - INFO - [LSTM] Epoch 11/20, Train Loss: 0.0001
2025-11-11 00:15:38,088 - INFO - [LSTM] Epoch 12/20, Train Loss: 0.0001
2025-11-11 00:15:39,738 - INFO - [LSTM] Epoch 13/20, Train Loss: 0.0000
2025-11-11 00:15:41,386 - INFO - [LSTM] Epoch 14/20, Train Loss: 0.0000
2

In [9]:
# Evaluation
# Đánh giá trên tập test
model.eval()
y_true = []
y_pred = []
with torch.no_grad():
    for batch_features, batch_labels in test_dataloader:
        batch_features = batch_features.to(device)
        outputs = model(batch_features)
        _, predicted = torch.max(outputs, 1)
        y_true.extend(batch_labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())
accuracy = 100 * sum(np.array(y_true) == np.array(y_pred)) / len(y_true)
logging.info(f"Final Accuracy on test data: {accuracy:.2f}%")

# Báo cáo phân loại
report = classification_report(y_true, y_pred, target_names=[id2label[str(i)] for i in range(len(id2label))])
print("\nBáo cáo phân loại trên tập test:")
print(report)

# Tính confusion matrix
cm = confusion_matrix(y_true, y_pred)


2025-11-11 00:23:32,199 - INFO - [LSTM] Final Accuracy on test data: 91.89%



Báo cáo phân loại trên tập test:
              precision    recall  f1-score   support

   Bhujasana       1.00      0.95      0.97        99
  Padamasana       0.96      0.98      0.97        87
    Tadasana       1.00      0.81      0.90        90
   Trikasana       0.86      1.00      0.93        87
 Vrikshasana       0.70      0.80      0.74        44

    accuracy                           0.92       407
   macro avg       0.90      0.91      0.90       407
weighted avg       0.93      0.92      0.92       407



In [ ]:
# Visualization
# Biểu đồ loss
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(train_losses) + 1), train_losses, marker='o', color='b', label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss qua các Epoch')
plt.legend()
plt.grid(True)
plt.show()

# Biểu đồ accuracy
plt.figure(figsize=(10, 6))
plt.bar(['Test Accuracy'], [accuracy], color='g')
plt.ylabel('Accuracy (%)')
plt.title('Accuracy trên tập test')
plt.ylim(0, 100)
plt.show()

# Biểu đồ confusion matrix
plt.figure(figsize=(10, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=[id2label[str(i)] for i in range(len(id2label))],
             yticklabels=[id2label[str(i)] for i in range(len(id2label))])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# Biểu đồ precision, recall, f1-score
report_dict = classification_report(y_true, y_pred, target_names=[id2label[str(i)] for i in range(len(id2label))], output_dict=True)
classes = list(id2label.values())
precision = [report_dict[cls]['precision'] for cls in classes]
recall = [report_dict[cls]['recall'] for cls in classes]
f1 = [report_dict[cls]['f1-score'] for cls in classes]

x = np.arange(len(classes))
width = 0.25

plt.figure(figsize=(12, 6))
plt.bar(x - width, precision, width, label='Precision', color='skyblue')
plt.bar(x, recall, width, label='Recall', color='lightgreen')
plt.bar(x + width, f1, width, label='F1-Score', color='salmon')
plt.xlabel('Classes')
plt.ylabel('Scores')
plt.title('Precision, Recall, and F1-Score per Class')
plt.xticks(x, classes, rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

# Biểu đồ phân bố nhãn train
plt.figure(figsize=(10, 6))
plt.bar(train_distribution.keys(), train_distribution.values(), color='lightblue')
plt.xlabel('Classes')
plt.ylabel('Number of Samples')
plt.title('Distribution of Labels in Train Set')
plt.xticks(rotation=45)
plt.show()

# Biểu đồ phân bố nhãn test
plt.figure(figsize=(10, 6))
plt.bar(test_distribution.keys(), test_distribution.values(), color='lightcoral')
plt.xlabel('Classes')
plt.ylabel('Number of Samples')
plt.title('Distribution of Labels in Test Set')
plt.xticks(rotation=45)
plt.show()

# Lưu mô hình
torch.save(model.state_dict(), 'models/yoga_classifier.pth')
logging.info("Mô hình đã được lưu tại models/yoga_classifier.pth")


In [ ]:
# Full Prediction on Test Data
# Hàm dự đoán một mẫu
def predict_sample(model, features, id2label, device):
    model.eval()
    with torch.no_grad():
        features = torch.tensor(features, dtype=torch.float32).unsqueeze(0).to(device)
        output = model(features)
        _, predicted = torch.max(output, 1)
        return id2label[str(predicted.item())]

# Dự đoán toàn bộ dữ liệu test
y_true_full = []
y_pred_full = []
for i in range(len(test_features)):
    true_label = id2label[str(test_labels[i])]
    pred_label = predict_sample(model, test_features[i], id2label, device)
    y_true_full.append(true_label)
    y_pred_full.append(pred_label)
    # In 5 mẫu đầu tiên để kiểm tra
    if i < 5:
        print(f"Mẫu test {i}: Dự đoán = {pred_label}, Thực tế = {true_label}")

# Tính toán số dự đoán đúng và accuracy
correct_predictions = sum(1 for true, pred in zip(y_true_full, y_pred_full) if true == pred)
total_samples = len(y_true_full)
accuracy_full = (correct_predictions / total_samples) * 100

# In kết quả thống kê
print(f"\nTổng số mẫu trong tập test: {total_samples}")
print(f"Số dự đoán đúng: {correct_predictions}")
print(f"Accuracy trên toàn bộ tập test: {accuracy_full:.2f}%")
